In [1]:
import os 
import pandas as pd 


import regex as re
import unicodedata
import spacy
import nltk
from nltk.probability import FreqDist
import numpy as np
from nltk.stem import WordNetLemmatizer

from tqdm.auto import tqdm, trange
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import os 

from langdetect import detect

C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://num

In [2]:
sales = pd.read_excel ('sales/sales.xlsx')

In [3]:
sales.columns

Index(['Month', 'WEEK NUM', 'DATE', 'TYPE', 'TENURE TP', 'CCMS ', 'SUPERVISOR',
       'AGENT', 'INT TP', 'CHATS Hopper', 'SALES TP', 'SALES Hopper', 'CR TP',
       'CLIENT CR', 'Day', 'Wave ', 'COMPARISON', 'AUX'],
      dtype='object')

In [4]:
#Sales report 
columns_needed = ['DATE', 'CCMS ',
       'CHATS Hopper', 'SALES TP',
       'SUPERVISOR', 'AGENT']

# Se que no es una buena practica poner el supervisor y a gente pero es la unica, porque las jerarquias cambian con el tiempo.
#
#        
sales = pd.read_excel ('sales/sales.xlsx', usecols=columns_needed)


sales.rename(columns={'DATE': 'Date', 'CCMS ':'CCMS', 'CHATS Hopper':'Chats', 'SALES TP':'Sales','SUPERVISOR':'Supervisor',
                             'AGENT':'Agent'}, inplace=True)

sales['Date']=pd.to_datetime(sales['Date']).dt.date
#sales.dropna(subset=['CCMS'],inplace=True)
sales['CCMS']=sales['CCMS'].apply(round)


sales.to_csv('output/sales.csv', index=False, encoding='utf8')
sales.head()

,Date,CCMS,Supervisor,Agent,Chats,Sales
0,2023-03-01,4694917,Francisco Romano Ruiz,Adele Baudouin Diaz,5,0
1,2023-03-01,5009536,Francisco Romano Ruiz,Adrián Marcelo Mora Chuquillanqui,0,0
2,2023-03-01,5048166,Luz Salazar Chavez,Alexa Gabriela Obando Campos,0,0
3,2023-03-01,4382854,Luis Guzman Reyes,Alexandra Pamela Broncano Broncano,1,0
4,2023-03-01,4846712,Solange Olivares,Alfredo Perez Diaz,56,8


In [5]:
sales

,Date,CCMS,Supervisor,Agent,Chats,Sales
0,2023-03-01,4694917,Francisco Romano Ruiz,Adele Baudouin Diaz,5,0
1,2023-03-01,5009536,Francisco Romano Ruiz,Adrián Marcelo Mora Chuquillanqui,0,0
2,2023-03-01,5048166,Luz Salazar Chavez,Alexa Gabriela Obando Campos,0,0
3,2023-03-01,4382854,Luis Guzman Reyes,Alexandra Pamela Broncano Broncano,1,0
4,2023-03-01,4846712,Solange Olivares,Alfredo Perez Diaz,56,8
...,...,...,...,...,...,...
11602,2023-06-13,5058300,Luz Salazar Chavez,Pamela Pastruso Leon,14,3
11603,2023-06-13,4941478,Luz Salazar Chavez,Nadya Juliet Orellana Pazimiño,11,3
11604,2023-06-13,4846715,Luz Salazar Chavez,Angelo Moran Tarrillo,0,0
11605,2023-06-13,4988565,Luz Salazar Chavez,Diego Sebastian Espinoza Estrada,23,1


In [6]:
sales['CCMS'].isna().sum()

0

In [7]:
sales['CCMS'].isna().sum()

0

In [8]:
sales['CCMS']

0        4694917
1        5009536
2        5048166
3        4382854
4        4846712
          ...   
11602    5058300
11603    4941478
11604    4846715
11605    4988565
11606    4461998
Name: CCMS, Length: 11607, dtype: int64

In [9]:
filename = ('TPClient/'+os.listdir('TPClient')[0])
columns_needed = ['Customer Name','Contact Method 1','Contact Method 2']

tpclient = pd.read_excel (filename, skiprows=7, usecols=columns_needed)

tpclient=tpclient.rename(columns={
                            'Contact Method 1':'TCD 1',
                            'Contact Method 2':'TCD 2',
                            })
tpclient['Customer Name']=tpclient['Customer Name'].apply(lambda x: str(x).replace(' ',''))

tpclient

,Customer Name,TCD 1,TCD 2
0,nan,NaN,
1,https://hopper.kustomerapp.com/app/customers/6...,Customer Support,Price Freeze -- Hopper Wallet
2,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- Hotel Options
3,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- Hotel Options
4,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- Hotel Options
...,...,...,...
55607,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- Hotel Options
55608,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- 15% Off
55609,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- Hotel Options
55610,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- Hotel Options


In [10]:
# Head Count 
columns_needed = [  'Agent',
                    'Supervisor',
                    'TP E-MAIL',
                    'Tenure Clasif',
                    'CCMS ID '
                    ]
hc = pd.read_excel('HC/HC.xlsx', usecols=columns_needed, sheet_name='HC_Active', skiprows=1)

hc.dropna(subset=['TP E-MAIL'], inplace=True)
hc['TP E-MAIL'] = hc['TP E-MAIL'].apply(lambda x: x.lower())

hc.to_csv('output/HeadCount.csv', encoding='utf8', index=False)

In [11]:
hc.head()

,CCMS ID,Agent,Supervisor,TP E-MAIL,Tenure Clasif
0,4967032,Cinthia Medina Vélez,Sebastian Rojas Fernandez,cinthia.medinavelez@teleperformance.com,NaN
1,3434150,Manuela Valencia Ospina,Cinthia Medina Vélez,manuela.valenciaospina@teleperformance.com,NaN
4,4539035,Luis Guzman Reyes,Manuela Valencia Ospina,luis.guzmanreyes@teleperformance.com,NaN
5,4160568,Luz Salazar Chavez,Manuela Valencia Ospina,luz.salazarchavez@teleperformance.com,+180
6,3875959,Solange Olivares León,Manuela Valencia Ospina,solange.olivaresleon@teleperformance.com,+180


In [12]:
files = os.listdir('input')
data_path = 'input/'+os.listdir('input')[0]
#df=pd.read_excel(data_path, sheet_name='Crudo Surveys', engine='openpyxl')
df=pd.read_excel(data_path)

In [13]:
columns_needed = [  'request_id',
                    'response_received_at',
                    'star_rating',
                    'comments',
                    'areas_for_improvement',
                    'employee_email',
                    'nps_response',
                    'nps_comment',
                    'channel',
                    'language',
                    'Was this a sale?',
                    'external_url'
                    ]



df2=df[columns_needed]
df2=df2.rename(columns={    'response_received_at':'Date',
                            'request_id':'ID',
                            'star_rating':'CSAT Response',
                            'comments':'CSAT Comment',
                            'areas_for_improvement':'CSAT Areas of improvement',
                            'employee_email': 'Employee Email',
                            'nps_response' : 'NPS Response',
                            'nps_comment' : 'NPS Comment',
                            'language' : 'Language',
                            'Was this a sale?' : 'Sale',
                            'external_url':'Customer Name',
                            'channel':'LOB'
                            })

df2 = df2.merge(tpclient,on='Customer Name', how='left')


def get_promoter(kpi):
    if kpi >= 4:
        return "Promoter"
    elif kpi >= 3:
        return "Passive"
    elif kpi >= 0:
        return "Detractor"

#Just english -- spanish surveys aprox 0.1% 
df2=df2[df2['Language'] == 'en']

#Get Promoter
df2['CSAT'] = df2['CSAT Response'].apply(lambda x: get_promoter(x))


#Get just date 
df2['Date']=pd.to_datetime(df2['Date']).dt.date

# Get the NPS like an integer 
df2['NPS Response']=df2['NPS Response'].apply(lambda x: str(x).split('.')[0])

#Decode correctly 
def decoding(x):
    if type(x) == str:
        return x.encode('Windows-1252', errors='ignore').decode('utf-8', errors='ignore')
    else:
        return x

df2['CSAT Comment'] = df2['CSAT Comment'].apply(lambda x: decoding(x))
df2.drop_duplicates(subset=['ID'], inplace=True)
#Save 
df2.to_csv('output/raw_surveys.csv', index=False, encoding='utf8')




In [14]:
df2

,ID,Date,CSAT Response,CSAT Comment,CSAT Areas of improvement,Employee Email,NPS Response,NPS Comment,LOB,Language,Sale,Customer Name,TCD 1,TCD 2,CSAT
0,795be8de-9897-4821-b027-0fbb3767d4df,2023-01-01,4,NaN,NaN,humberto.brignolepardo@teleperformance.com,nan,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
1,e5d5da0f-3bfa-4647-a9b2-6aed3ffeebc8,2023-01-02,3,NaN,"[""Friendliness""]",humberto.brignolepardo@teleperformance.com,6,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Passive
2,77cb8a94-3904-4162-82d2-a861f59ef281,2023-01-02,5,NaN,NaN,jorge.riveraparedes@teleperformance.com,nan,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
3,5f9fbec6-3473-4501-a0de-f61fdb4c084c,2023-01-02,5,Replied very fast found me exactly what I was ...,NaN,humberto.brignolepardo@teleperformance.com,10,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
4,eedf1ff7-e0a3-478c-b719-bb94706ce1f5,2023-01-03,1,NaN,NaN,frida.nunezhuarancca@teleperformance.com,nan,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Detractor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5735,d7b3b726-b5ce-4f21-add4-4092ab7a1dd7,2023-06-13,5,Attentive And precise,NaN,jorge.castroaguirre@teleperformance.com,9,NaN,Phone,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- 15% Off,Promoter
5736,d19a6480-f2d8-4187-8e41-16d503c9f315,2023-06-13,4,NaN,NaN,ismael.salazaryanaco@teleperformance.com,nan,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
5737,d1d0720e-2b42-4aa8-baae-ba5f153d0261,2023-06-13,4,NaN,NaN,naylin.ruizvila@teleperformance.com,8,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
5738,acdcb3ff-9b0f-436b-b6cf-c8ea0ccebdad,2023-06-13,5,He was able to get back to me in minutes and l...,NaN,luis.padillahuerta@teleperformance.com,10,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,Customer Support,Hotel -- Voided Payment,Promoter


In [15]:
df2

,ID,Date,CSAT Response,CSAT Comment,CSAT Areas of improvement,Employee Email,NPS Response,NPS Comment,LOB,Language,Sale,Customer Name,TCD 1,TCD 2,CSAT
0,795be8de-9897-4821-b027-0fbb3767d4df,2023-01-01,4,NaN,NaN,humberto.brignolepardo@teleperformance.com,nan,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
1,e5d5da0f-3bfa-4647-a9b2-6aed3ffeebc8,2023-01-02,3,NaN,"[""Friendliness""]",humberto.brignolepardo@teleperformance.com,6,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Passive
2,77cb8a94-3904-4162-82d2-a861f59ef281,2023-01-02,5,NaN,NaN,jorge.riveraparedes@teleperformance.com,nan,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
3,5f9fbec6-3473-4501-a0de-f61fdb4c084c,2023-01-02,5,Replied very fast found me exactly what I was ...,NaN,humberto.brignolepardo@teleperformance.com,10,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
4,eedf1ff7-e0a3-478c-b719-bb94706ce1f5,2023-01-03,1,NaN,NaN,frida.nunezhuarancca@teleperformance.com,nan,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Detractor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5735,d7b3b726-b5ce-4f21-add4-4092ab7a1dd7,2023-06-13,5,Attentive And precise,NaN,jorge.castroaguirre@teleperformance.com,9,NaN,Phone,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,Sale,Hotels -- 15% Off,Promoter
5736,d19a6480-f2d8-4187-8e41-16d503c9f315,2023-06-13,4,NaN,NaN,ismael.salazaryanaco@teleperformance.com,nan,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
5737,d1d0720e-2b42-4aa8-baae-ba5f153d0261,2023-06-13,4,NaN,NaN,naylin.ruizvila@teleperformance.com,8,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,NaN,NaN,Promoter
5738,acdcb3ff-9b0f-436b-b6cf-c8ea0ccebdad,2023-06-13,5,He was able to get back to me in minutes and l...,NaN,luis.padillahuerta@teleperformance.com,10,NaN,Email,en,NaN,https://hopper.kustomerapp.com/app/customers/6...,Customer Support,Hotel -- Voided Payment,Promoter


In [16]:
df_areas = df2[['ID', 'CSAT Areas of improvement']]
df_areas.dropna(subset=['CSAT Areas of improvement'], inplace=True)
df_areas['CSAT Areas of improvement'] = df_areas['CSAT Areas of improvement'].apply(lambda x: [word.strip('" []') for word in x.split(',')])
df_areas = df_areas.explode('CSAT Areas of improvement')

df_areas.to_csv('output/areas_improvement.csv', index=False, encoding='utf8')

C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
df = df2[['ID', 'CSAT Comment']]

df.dropna(subset=['CSAT Comment'], inplace=True)


df_sentiment= df.copy()

nlp_en = spacy.load('en_core_web_md')

df['CSAT Comment']=df['CSAT Comment'].apply(lambda x: str(x).lower())

#Remove special characters

special_chars1 = df['CSAT Comment'].apply(lambda x: [each for each in list(x) if not each.isalnum() and each != ' '] if type(x) == type('Hello') else '')
flat_list1 = [item for sublist in special_chars1 for item in sublist]
print(set(flat_list1)) # Set = List of character with no repeat
special_char_to_remove = set(flat_list1)

def remove_special_char(words):
    return [word for word in words if not str(word) in special_char_to_remove]

df['spacy']=df['CSAT Comment'].apply(lambda x: remove_special_char(nlp_en(str(x))))

df['Lemmatization'] = df['spacy'].apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space and len(token) > 2 and token.is_alpha]) # doc if str(doc) == 'nan' else # not token.is_stop and



df_cat = df.copy()

df_cat['joint_lemma_words'] = df_cat['Lemmatization'].apply(lambda x: ' '.join(x))
df_cat.drop(['Lemmatization', 'CSAT Comment','spacy'], axis = 1, inplace = True)

df.to_csv("output/freqwords.csv",index=False, sep=';', encoding='utf-8')

df_copy=df.copy()


C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


{'💰', '🙂', '£', '?', '🎩', '👌', '…', ';', '‘', '🙌', '-', '“', '’', "'", '#', '&', '🙄', '\U0001f90e', '😊', ':', '♥', '😔', '☺', ',', '👊', '😎', '.', '+', ')', '😉', '\U0001faf6', '😞', '%', '"', '😈', '😇', '@', '\n', '(', '🥰', '$', '/', '!', '😡', '*'}


C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
# Drop 'Feedback' and 'spacy' columns
df.drop(['CSAT Comment', 'spacy'], axis=1, inplace=True)

#Super Important! 
df.reset_index(drop=True, inplace=True)

df_bigram=df.copy()
df_trigram=df.copy()

df.head()

C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID,Lemmatization
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,"[reply, fast, find, exactly, look, kind]"
1,4ad38380-bdbd-4045-953f-5319e95b346d,"[polite, want, ask, guy, end]"
2,71251b25-c385-4a53-8a5d-165fe4e44b02,[helpful]
3,635601ee-085c-4203-af81-0e91f5b43064,"[help, problem]"
4,36c15d6e-bd58-433d-99b4-3708a03ebd6b,[chat]


In [19]:
df["frecwords"]=df['Lemmatization'].apply(lambda x: FreqDist(x))
df=df.drop(['Lemmatization'], axis = 1)

df_temp = pd.DataFrame(pd.DataFrame(df["frecwords"].values.tolist()).stack().reset_index(level=1))
df_temp.columns = ['Words', 'Frequency']
df= df.join(df_temp)

df.drop(["frecwords"], axis = 1, inplace = True)

# Removing unnecesary blankspaces around words
df['Words'] = df['Words'].apply(lambda x: str(x).strip())

df.dropna(inplace=True)
df['Frequency'] = df['Frequency'].astype('int8')
df.to_csv(r"output/Words.csv", encoding = 'utf-8', sep=';', index=False)

df.head()

C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Words,Frequency
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,reply,1
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,fast,1
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,find,1
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,exactly,1
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,look,1


In [20]:

## Bigrams 

def common_bigrams(words_list):
    list_bigrams = list(nltk.bigrams(words_list))
    filtered_bigram_dist = FreqDist(list_bigrams)
    return dict(filtered_bigram_dist)

df_bigram['bigrams_dict'] = df_bigram['Lemmatization'].apply(lambda x: common_bigrams(x))

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
df_temp_bigrams = pd.DataFrame(pd.DataFrame(df_bigram['bigrams_dict'].values.tolist()).stack().reset_index(level=1))
df_temp_bigrams.columns = ['Bigrams_tuple', 'bigrams_count']

display(df_temp_bigrams)
# Split bigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_temp_bigrams[['bigram_1', 'bigram_2']] = pd.DataFrame(df_temp_bigrams['Bigrams_tuple'].tolist(), index=df_temp_bigrams.index)

df_bigram = df_bigram.join(df_temp_bigrams)

df_bigram.drop(['Lemmatization', 'bigrams_dict','Bigrams_tuple' ], axis = 1, inplace = True)
df_bigram['Bigrams'] = df_bigram['bigram_1'] +" " +df_bigram['bigram_2']
df_bigram.dropna(subset=['bigrams_count'], inplace=True)
df_bigram['bigrams_count'] = df_bigram['bigrams_count'].astype('int8')
df_bigram.to_csv("output/Bigrams.csv", sep=';', encoding='utf-8', index=False)
df_bigram.head()

,Bigrams_tuple,bigrams_count
0,"(reply, fast)",1.0
0,"(fast, find)",1.0
0,"(find, exactly)",1.0
0,"(exactly, look)",1.0
0,"(look, kind)",1.0
...,...,...
2412,"(include, refund)",1.0
2412,"(refund, able)",1.0
2412,"(able, understand)",1.0
2412,"(understand, help)",1.0


,ID,bigrams_count,bigram_1,bigram_2,Bigrams
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,reply,fast,reply fast
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,fast,find,fast find
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,find,exactly,find exactly
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,exactly,look,exactly look
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,look,kind,look kind


In [21]:
# Trigrams

def common_trigrams(words_list):
    list_trigrams = list(nltk.trigrams(words_list))
    filtered_trigrams_dist = FreqDist(list_trigrams)
    return dict(filtered_trigrams_dist)


df_trigram['trigrams_dict'] = df_trigram['Lemmatization'].apply(lambda x: common_trigrams(x))

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
df_temp_trigrams = pd.DataFrame(pd.DataFrame(df_trigram['trigrams_dict'].values.tolist()).stack().reset_index(level=1))
df_temp_trigrams.columns = ['Trigrams_tuple', 'Trigrams_count']

display(df_temp_trigrams)

df_temp_trigrams[['trigram_1', 'trigram_2', 'trigram_3']] = pd.DataFrame(df_temp_trigrams['Trigrams_tuple'].tolist(), index=df_temp_trigrams.index)

df_trigram = df_trigram.join(df_temp_trigrams)

df_trigram.drop(['Lemmatization', 'trigrams_dict','Trigrams_tuple' ], axis = 1, inplace = True)
df_trigram['Trigrams'] = df_trigram['trigram_1'] +" " +df_trigram['trigram_2'] +" " +df_trigram['trigram_3']
df_trigram.dropna(subset=['Trigrams_count'], inplace=True)
df_trigram['Trigrams_count'] = df_trigram['Trigrams_count'].astype('int8')
df_trigram.to_csv("output/Trigrams.csv", sep=';', encoding='utf-8', index=False)
df_trigram.head()

,Trigrams_tuple,Trigrams_count
0,"(reply, fast, find)",1.0
0,"(fast, find, exactly)",1.0
0,"(find, exactly, look)",1.0
0,"(exactly, look, kind)",1.0
1,"(polite, want, ask)",1.0
...,...,...
2412,"(explain, include, refund)",1.0
2412,"(include, refund, able)",1.0
2412,"(refund, able, understand)",1.0
2412,"(able, understand, help)",1.0


,ID,Trigrams_count,trigram_1,trigram_2,trigram_3,Trigrams
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,reply,fast,find,reply fast find
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,fast,find,exactly,fast find exactly
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,find,exactly,look,find exactly look
0,5f9fbec6-3473-4501-a0de-f61fdb4c084c,1,exactly,look,kind,exactly look kind
1,4ad38380-bdbd-4045-953f-5319e95b346d,1,polite,want,ask,polite want ask


In [22]:
# Sentiment 


######################### Model settings ############################
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
######################################################################

######################### Use model #################################
def get_sentiment_scores(tweet):
    encoded_input = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores
######################################################################

########################################### Get sentiment #######################################################
tokenizer.model_max_length = 1028
lst_negative_scores = []
lst_neutral_scores = []
lst_positive_scores = []
lst_label = []

df_sentiment['CSAT Comment'] = df_sentiment['CSAT Comment'].astype('str')
df_sentiment=df_sentiment[df_sentiment['CSAT Comment'].str.len() < 1500]

for i in trange(df_sentiment.shape[0]):
    tweet_i = str(df_sentiment['CSAT Comment'].values[i])
    
    scores_array = get_sentiment_scores(tweet_i)
    negative_score, neutral_score, positive_score = scores_array
    lst_negative_scores.append(negative_score)
    lst_neutral_scores.append(neutral_score)
    lst_positive_scores.append(positive_score)
    
    ranking = np.argsort(scores_array)
    ranking = ranking[::-1]
    label = config.id2label[ranking[0]]
    lst_label.append(label)
    
df_sentiment['NEGATIVE_SCORE'] = lst_negative_scores
df_sentiment['NEUTRAL_SCORE'] = lst_neutral_scores
df_sentiment['POSITIVE_SCORE'] = lst_positive_scores
df_sentiment['LABEL'] = lst_label



df_sentiment['NEGATIVE_SCORE'] = df_sentiment['NEGATIVE_SCORE'].astype('str')
df_sentiment['NEGATIVE_SCORE'] = df_sentiment['NEGATIVE_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment['NEUTRAL_SCORE'] = df_sentiment['NEUTRAL_SCORE'].astype('str')
df_sentiment['NEUTRAL_SCORE'] = df_sentiment['NEUTRAL_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment['POSITIVE_SCORE']=df_sentiment['POSITIVE_SCORE'].astype('str')
df_sentiment['POSITIVE_SCORE'] = df_sentiment['POSITIVE_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment.drop(['CSAT Comment'], axis = 1, inplace = True)

df_sentiment.to_csv('output/Sentiment.csv', sep=';', index=False)
df_sentiment

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/2414 [00:00<?, ?it/s]

,ID,NEGATIVE_SCORE,NEUTRAL_SCORE,POSITIVE_SCORE,LABEL
3,5f9fbec6-3473-4501-a0de-f61fdb4c084c,"0,005999353","0,02635757","0,967643",positive
5,4ad38380-bdbd-4045-953f-5319e95b346d,"0,7683971","0,215089","0,016513806",negative
6,71251b25-c385-4a53-8a5d-165fe4e44b02,"0,014199242","0,03601757","0,9497832",positive
7,635601ee-085c-4203-af81-0e91f5b43064,"0,8349163","0,15512788","0,00995579",negative
8,36c15d6e-bd58-433d-99b4-3708a03ebd6b,"0,4365304","0,53542316","0,02804648",neutral
...,...,...,...,...,...
5731,e4299863-af8c-4029-b7c4-41ca677f53b2,"0,0042784745","0,045621905","0,9500996",positive
5734,23942a12-5ffb-437f-99ba-8fa7b568b9a1,"0,90464485","0,08766063","0,0076944088",negative
5735,d7b3b726-b5ce-4f21-add4-4092ab7a1dd7,"0,036707092","0,3883646","0,5749283",positive
5738,acdcb3ff-9b0f-436b-b6cf-c8ea0ccebdad,"0,013757082","0,1114288","0,87481415",positive


In [23]:
### Categorization

def remove_nonalpha(words):
    list_nonalpha = [w.strip() for w in words if w.strip().isalpha()]        
    return list_nonalpha 
    
def get_category(x):
    key_list = []
    match_list = []
    for key in categories_dict.keys():
        for value in categories_dict[key]:
            if (bool(re.search(value,x)) ==True):
                if key not in key_list:
                    key_list.append(key)
                    match_list.append(re.compile(value).findall(x))
    return dict(zip(key_list, match_list))

df_agent = df_cat.copy()
df_channel=df_cat.copy()

In [24]:
categories_dict = {



'Unprofessional' : [
    'disrespectful',
    'rude', 
    'impolite',
    'unprofessional',
    'dominant',
    'patronise',
    'ignore',
    'attitude'
],

'No Resolution':[
    'resolve',
    'helpful',
    '(resolve)(\s|\w+\s|\s\w+\s){0,3}(issue)',
    'great'
],

'Knowledge':[
    '(tell)(\s|\w+\s|\s\w+\s)(look|search)(\s|\w+\s|\s\w+\s){0,3}(price)',
    'knowledg',
    '(wrong|irrelevant|false)',
    'confident',
    'quick'
],

'Wrong option':[
    '(bad)(\s|\w+\s|\s\w+\s){0,3}(suggestion|conversation)',
    '(come)(\s|\w+\s|\s\w+\s){0,3}(high)(\s|\w+\s|\s\w+\s){0,3}(rate)',
    '(answer)(\s|\w+\s|\s\w+\s){0,3}(correspond)(\s|\w+\s|\s\w+\s){0,3}(question)',
    '(provide|hotel|give|make|appropriate)(\s|\w+\s|\s\w+\s){0,3}(recommendation|suggestion)',
    '(recommendation)(\s|\w+\s|\s\w+\s){0,3}(offer|receive|price|range)'    

], 
'Friendly':[
    'friendly',
    'patience',
    'attentive',
    'professional',
    'proacive',
    'nice',
    'happy',
    'amazing'],

'Chat Handling':[
    '(good|receive|great|bad)(\s|\w+\s|\s\w+\s){0,3}(guidance)',
    '(call|return)(\s|\w+\s|\s\w+\s){0,3}(back|help|complete|call)'

],
'English Skills':['(speak|talk|understad|bad|poor|terrible|horrible)(\s|\w+\s|\s\w+\s){0,3}(english|language|clear)',
        '(hard|difficult|couldn)(\s|\w+\s|\s\w+\s){0,3}(understand)','(hard|heavy)(\s|\w+\s|\s\w+\s){0,3}(accent)',
        '(language)(\s|\w+\s|\s\w+\s){0,3}(barrier)','(listen|understand|read)(\s|\w+\s|\s\w+\s){0,3}(issue|problem|concern|case|complain|question|message)',
        '(good|bad|wrong)(\s|\w+\s|\s\w+\s){0,3}(listen)',
        '(english)(\s|\w+\s|\s\w+\s){0,3}(no |not |wasn)(\s|\w+\s|\s\w+\s){0,3}(language)',
        '(speak|understad|bad|poor|terrible|horrible|break|comprehend|write|read)(\s|\w+\s|\s\w+\s){0,3}(english|language)',
        '(hard|difficult|couldn|imposible)(\s|\w+\s|\s\w+\s){0,3}(understand)',
        '(understand)(\s|\w+\s|\s\w+\s){0,3}(him|her|rep|agent|english|accent|each)',
        '(hard|heavy|strong|foreign|break|latin|spanish)(\s|\w+\s|\s\w+\s){0,3}(accent)',
        '(hard|difficult|couldn|impossible)(\s|\w+\s|\s\w+\s){0,3}(understand)(\s|\w+\s|\s\w+\s){0,3}(she|he)(\s|\w+\s|\s\w+\s){0,3}(say)',
        '(language)(\s|\w+\s|\s\w+\s){0,3}(barrier)','(english)(\s|\w+\s|\s\w+\s){0,3}(first|second)(\s|\w+\s|\s\w+\s){0,3}(language)', 
        '(english)(\s|\w+\s|\s\w+\s){0,3}(poor|bad|break|horrible|terrible)']

}




In [25]:
def get_category(x):
    key_list = []
    match_list = []
    for key in categories_dict.keys():
        for value in categories_dict[key]:
            if (bool(re.search(value,x)) ==True):
                if key not in key_list:
                    key_list.append(key)
                    match_list.append(re.compile(value).findall(x))
    return dict(zip(key_list, match_list))




In [26]:
df_agent['cat_agent_dict'] = df_agent["joint_lemma_words"].apply(lambda x:get_category(x))

df_agent["dict_len"] = df_agent["cat_agent_dict"].apply(lambda x: len(x))
df_nohits1= df_agent[df_agent["dict_len"] == 0]
df_agent = df_agent[df_agent["dict_len"] > 0]

df_agent.reset_index(inplace = True, drop=True)

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
#df_temp_cat_agent = 
df_agent_temp = pd.DataFrame(pd.DataFrame(df_agent['cat_agent_dict'].values.tolist()).stack().reset_index(level=1))
#df_temp_bigrams.columns = ['Bigrams', 'bigrams_count']

df_agent_temp.columns = ['category', 'hit']
#df_temp_bigrams

df_agent_temp['hit'] = df_agent_temp['hit'].apply(lambda x: remove_nonalpha(list(x[0])))

# Split trigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_agent_temp = pd.concat([df_agent_temp, pd.DataFrame(df_agent_temp['hit'].tolist(), index=df_agent_temp.index)], axis=1)
df_agent_temp['hit'] = df_agent_temp['hit'].apply(lambda x : ' '.join(x))

df_agent = df_agent.join(df_agent_temp)

df_agent.drop(['cat_agent_dict', 'dict_len'], axis = 1, inplace = True)
df_agent['categorization_level'] = 'Agent'
df_agent.head()

,ID,joint_lemma_words,category,hit,0,1,2,3,4,5,6,7,8,9,10,11,12,13,categorization_level
0,71251b25-c385-4a53-8a5d-165fe4e44b02,helpful,No Resolution,h e l p f u l,h,e,l,p,f,u,l,None,None,None,None,None,None,None,Agent
1,da5db9f6-13dd-4c4e-8725-cdb95fa6aad6,nice,Friendly,n i c e,n,i,c,e,None,None,None,None,None,None,None,None,None,None,Agent
2,bc233b62-0072-4a20-9bbe-b34d17691275,helpful,No Resolution,h e l p f u l,h,e,l,p,f,u,l,None,None,None,None,None,None,None,Agent
3,4fc2dda8-9ea2-4b1b-8558-b2bda98859f3,professional understand flight get error say f...,Friendly,p r o f e s s i o n a l,p,r,o,f,e,s,s,i,o,n,a,l,None,None,Agent
4,806c11b9-5d86-4265-8ca6-b8e7cc042b1d,nice helpful thank,No Resolution,h e l p f u l,h,e,l,p,f,u,l,None,None,None,None,None,None,None,Agent


In [27]:
categories_dict = {
'Robotic Script': [
    r"(?:slow|delay) service",
    r"robot(?:ic)?",
    r"\bchatbot\b",
    '(response|message|receive|conversation|answer)(\s|\w+\s|\s\w+\s){0,3}(automated?)',
    '(record(ed)?|automate(d)?)(\s|\w+\s|\s\w+\s){1,3}(response(s)?|answer|reply|conversation)', 
    '(able|definitely)(\s|\w+\s|\s\w+\s){0,3}(talk|real)(\s|\w+\s|\s\w+\s){0,3}(person)', 
    '(live|real)(\s|\w+\s|\s\w+\s){0,3}(person|agent)'
],

'Time' : ['(chat)(\s|\w+\s|\s\w+\s){1,3}(wait|hour|minute|forever)'
    '(forever|slow|delay)(\s|\w+\s|\s\w+\s){1,3}(connect|response|reply|answer)',
    '(take|wait|respond|waste|resolve|contact|speak|response)(\s|\w+\s|\s\w+\s){1,3}(long|slow|forever|delay|support|speak|help|hour|later|time)',
    '(tired)(\s|\w+\s|\s\w+\s){1,3}(wait)',
    'long time',
    '(forever|hour|minute|delay|long|slow)(\s|\w+\s|\s\w+\s){1,3}(answer|take|reply|contact|speak|wait|respond|connect|response|support|resolve|refund|help|later)',
    '(impossible)(\s|\w+\s|\s\w+\s){1,3}(find|contact)'
],

'Refund' : [
    'refund',
    '(money)(\s|\w+\s|\s\w+\s){0,3}(back|right|away|account)',
    '(return|want)(\s|\w+\s|\s\w+\s){0,3}(money)'
    ],

'False Advertising': [
    '(false)(\s|\w+\s|\s\w+\s){0,3}(advertis(ing)?)',
    'scam',
    '(secret)(\s|\w+\s|\s\w+\s){0,3}(deal)'
],
'Price freeze':
    ['(price)(\s|\w+\s|\s\w+\s){0,3}(freeze)'],

'App':[
    r"\bapp\b",
    'site'
    'web'
]}

In [28]:
df_channel['cat_agent_dict'] = df_channel["joint_lemma_words"].apply(lambda x: get_category(x))

df_channel["dict_len"] = df_channel["cat_agent_dict"].apply(lambda x: len(x))
df_nohits2= df_channel[df_channel["dict_len"] == 0]
df_channel = df_channel[df_channel["dict_len"] > 0]
df_channel.reset_index(inplace = True, drop=True)

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
#df_temp_cat_agent = 
df_channel_temp = pd.DataFrame(pd.DataFrame(df_channel['cat_agent_dict'].values.tolist()).stack().reset_index(level=1))
#df_temp_bigrams.columns = ['Bigrams', 'bigrams_count']

df_channel_temp.columns = ['category', 'hit']
#df_temp_bigrams

df_channel_temp['hit'] = df_channel_temp['hit'].apply(lambda x: remove_nonalpha(list(x[0])))

# Split trigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_channel_temp = pd.concat([df_channel_temp, pd.DataFrame(df_channel_temp['hit'].tolist(), index=df_channel_temp.index)], axis=1)
df_channel_temp['hit'] = df_channel_temp['hit'].apply(lambda x : ' '.join(x))

df_channel = df_channel.join(df_channel_temp)

df_channel.drop(['cat_agent_dict', 'dict_len'], axis = 1, inplace = True)
df_channel['categorization_level'] = 'Brand'
df_channel.head()

,ID,joint_lemma_words,category,hit,0,1,2,3,4,5,6,7,8,9,10,categorization_level
0,abda29e2-8d83-48ab-8626-ceba29dc93fe,need refund,Refund,r e f u n d,r,e,f,u,n,d,None,None,None,None,None,Brand
1,365db3a1-fb2a-4bbe-b32c-692450072b61,rep help try find deal app keep make appear ne...,App,a p p,a,p,p,None,None,None,None,None,None,None,None,Brand
2,3e6befe2-843c-4907-ae24-c12b335e979b,achieve turn hopper false advertisement,False Advertising,false advertis,false,advertis,None,None,None,None,None,None,None,None,None,Brand
3,fdcaf0bd-487c-4ccd-b7ac-5da0a8b5cd02,fast response polite helpful pick,Time,response polite help,response,polite,help,None,None,None,None,None,None,None,None,Brand
4,89fd709d-1b68-4891-bf9a-4aa18b175666,helpful answer question response time bit slow...,App,a p p,a,p,p,None,None,None,None,None,None,None,None,Brand


In [29]:
df_cat = pd.concat([df_agent, df_channel])
df_cat=df_cat[['ID', 'category', 'categorization_level']]

In [30]:

df_cat = df_cat.merge(df_sentiment,on= 'ID', how='left')
df_cat.to_csv('output/Categorization.csv', index=False, sep=';')

In [31]:
df_cat

,ID,category,categorization_level,NEGATIVE_SCORE,NEUTRAL_SCORE,POSITIVE_SCORE,LABEL
0,71251b25-c385-4a53-8a5d-165fe4e44b02,No Resolution,Agent,"0,014199242","0,03601757","0,9497832",positive
1,da5db9f6-13dd-4c4e-8725-cdb95fa6aad6,Friendly,Agent,"0,04087376","0,18506715","0,7740592",positive
2,bc233b62-0072-4a20-9bbe-b34d17691275,No Resolution,Agent,"0,8663512","0,11780255","0,015846232",negative
3,4fc2dda8-9ea2-4b1b-8558-b2bda98859f3,Friendly,Agent,"0,93159646","0,06286709","0,005536567",negative
4,806c11b9-5d86-4265-8ca6-b8e7cc042b1d,No Resolution,Agent,"0,0044302335","0,014392817","0,98117703",positive
...,...,...,...,...,...,...,...
1691,6d9f2b01-3a17-48f7-af68-7e47fe94df1f,Time,Brand,"0,006833723","0,058261767","0,9349043",positive
1692,e8c5d241-6983-49b6-b280-2aa7cbeac12a,App,Brand,"0,9442253","0,048811484","0,006963122",negative
1693,00b0b32a-99a4-47ed-8117-8e2734efc623,Time,Brand,"0,5769956","0,3790767","0,043927655",negative
1694,cd412d90-0fce-422f-8cc2-50aeacf36fa3,App,Brand,"0,5657982","0,4178817","0,016319958",negative
